# 14주차 (0603) Review
---
---

## 함수II
1. 일급 객체
2. 중첩 함수
    1. 클로저
    2. 데코레이터
    3. 범위
3. 익명 함수
4. 제너레이터
5. 재귀 함수

---
### 1. 일급 객체
- first class object, first class citizen
- 함수도 파이썬에서는 일급 객체
    - 다른 언어에서는 안 되는데, 파이썬에서는 함수도 이런 역할이 주어졌다는 것!
- 일급 객체의 조건
    - 함수의 인자로 전달된다.
    ```
        def fx(func)  # >>> 여기서 func는 `함수`
    ```
    - 함수의 반환값이 된다. 
    ```
        def fx(func):
            return func  # >>> 함수를 내보낼 수도 있다.
    ```
    - 수정, 할당이 된다.
    ```
        var = fx()
    ```

In [1]:
def answer():
    print(815)
    
def run_sth(func):
    func() # func vs. func() 차이! => ()가 붙으면, 실행을 하라는 의미
    
run_sth(answer) # 함수인 answer이 들어가서, run_sth이 실행이 됨!

815


In [2]:
def add_args(arg1, arg2, arg3):
    print(arg1 + arg2 + arg3)
    
def run_sth2(func, *args): # asterisk 사용
    func(*args)
    
run_sth2(add_args, 10, 20, 30)

60


---
### 2. 중첩함수
- 일급 객체 활용, 함수 내에서 또 다른 함수를 정의하는 것
- 내부함수 캡슐화
    - 사용하는 이유?
        1. 메모리 절약 (외부함수를 지워도, 내부함수를 돌아가게 할 수 있음)
        2. 변수가 섞여서 불필요하게 충돌하는 것을 방지함 (내용에 맞게 바운더리를 주는 역할)
        3. 목적에 맞게 변수를 그룹화할 수 있음. 관리, 책임 명확히!

In [3]:
# 중첩함수의 기본 형식

def outer(a, b): # 외부함수
    def inner(c, d): # 내부함수
        return c + d
    return inner(a, b)

outer(1, 1)

2

In [4]:
# 이렇게하면 Error 
# inner는 안에 있기 때문에, 밖에서 건드릴 수 없는 함수!
inner(1, 1) 

NameError: name 'inner' is not defined

In [5]:
c # 역시 Error

NameError: name 'c' is not defined

In [6]:
def dog(saying):
    def inner():
        return f'we are the dogs who say: {saying}'
    return inner

a = dog('arf')
b = dog('woof')

In [7]:
a, b # 함수

(<function __main__.dog.<locals>.inner()>,
 <function __main__.dog.<locals>.inner()>)

In [8]:
a(), b() # 실행시키고 싶다

('we are the dogs who say: arf', 'we are the dogs who say: woof')

In [9]:
def inner():
    return f'we are the knights who say: {saying}'

inner() # Error

NameError: name 'saying' is not defined

#### 여기서 알 수 있는 `inner 함수의 장점`
- 외부함수의 인자를 참조할 수 있다; saying을 inner에서 안 받았는데, 잘 출력이 된다.
- 수정/활용은 안 됨.
- 이런 게 바로 클로저, 중첩함수의 일부분.

#### A. 클로저 | closure
- 조건
    1. 중첩함수일 것
    2. 내부함수가 외부함수의 상태값을 참조할 것 
    3. 외부함수의 리턴값이 내부함수일 것
- 정의
    - 외부함수의 상태값을 기억하는 함수 (호출 시 사용가능)

In [10]:
def multiply(x):
    def inner(y):     # 조건 1.
        return x * y  # 조건 2.
    return inner      # 조건 3.

In [11]:
m = multiply(1)
n = multiply(2)

print(m) # 함수
print(n) # 함수
print(m(10), n(10)) # 함수 실행!

<function multiply.<locals>.inner at 0x7fc8a1c48940>
<function multiply.<locals>.inner at 0x7fc8a1c48160>
10 20


In [12]:
del(multiply) # 메모리에서 multiply 지우기
multiply

NameError: name 'multiply' is not defined

In [13]:
# multiply를 지워도 얘는 계속 돌아간다. 
# 외부함수는 지워졌지만, 객체를 따로 기억한다 -> 메모리 효율적 사용 가능
n(10)

20

#### `실습1`
- 클로저 함수 만들기
- 리턴값 * 리턴값 (예 - 8 * 8)

In [14]:
def add(a, b):
    return a + b

def square(func):
    def inner(a, b): # func의 인자가 둘 => inner에서 설정함.
        result = func(a, b)
        return result * result
    return inner

s = square(add)
s(2, 6)

64

#### B. 데코레이터
- 클로저 - 데코레이터 사용이 가능하다.
- 메인 함수에 또 다른 함수를 취해 반환할 수 있게 한다.
- 재사용성 높다.
- 가독성, 직관성 좋다.

In [15]:
@square
def plus(a, b):
    return a * b

plus(2, 6) # square를 탄다.

144

#### C. scope | 범위
- 전역: global
- 지역: local (참조만 가능)
- nonlocal

In [16]:
### 범위

x = 4            # global
def add(a, b):
    y = 8        # local
    return a + b

def square(func):
    # local
    def inner(a, b): 
        # nonlocal
        result = func(a, b)
        return result * result
    return inner

In [17]:
x = 4            # global
def outer(c):
    y = 8        # local
    def inner():
        # 여기까지도 c는 9였음.
        c = 999  # nonlocal
        return c
    return inner()

outer(9) # 결과는 999

999

In [18]:
x = 4            # global
def outer(c):
    y = 8        # local
    def inner():
        c *= 1   # Error
        return c
    return inner()

outer(9)

UnboundLocalError: local variable 'c' referenced before assignment

In [19]:
# 위 Error 해결 방법?

x = 4            # global
def outer(c):
    y = 8        # local
    def inner():
        # c의 범위를 바꿔주기!
        nonlocal c
        c *= 1
        return c
    return inner()

outer(9)

9

#### `실습2`
- fx1: spped, limit를 받아서 내 속도가 제한속도를 위반하는지 t/f
- fx2: 클로저, 초과할 경우 얼마나 초과하는지 프린트하는 함수
- 실행은 데코레이터로 fx1를 실행하면 되도록 만들기

In [20]:
def violate(func):
    def inner(speed, limit):
        if func(speed, limit):
            return f'{speed - limit}km/h 초과되었습니다.'
        return f'현재 속도는 {speed}km/h, 정상 속도입니다.'
    return inner

@violate
def is_speeding(speed, limit):
    return speed > limit

is_speeding(100, 80)

'20km/h 초과되었습니다.'

In [21]:
print(is_speeding(60, 70))
print(is_speeding(50, 50))

현재 속도는 60km/h, 정상 속도입니다.
현재 속도는 50km/h, 정상 속도입니다.


---
### 3. 익명함수 | lambda
- 이름이 없다. 예약어가 없다.
```
    def is_speeding():
        return
```
- def, return  => 이런 예약어를 안 씀!
- is_speeding  => 이름은 써도 되고, 안 써도 됨!
- 단순한 용도의 함수가 필요할 경우 사용, 재사용할 필요 없을 때.
- 잦은 사용은 권하지 않음. 직관적이지 않기 때문에!
- 형식 - lambda x: <x를 요리할 코드>

In [22]:
def add_one(x):
    return x + 1

print(add_one(5))

# 위의 def와 같은 코드를 한 줄로, 번거로움을 줄일 수 있다
print((lambda x: x + 1)(5))

6
6


In [23]:
# 이름을 만들어 주는 방법
fx = lambda x: x + 1
fx(5)

6

In [24]:
# 변수를 여러 개 사용하고 싶으면 ,로 가져가면 됨
f = lambda x, y: x + y
f(5, 10)

15

#### `실습3`
- 단어가 들어왔을 때 첫 글자를 대문자로 바꾸고 단어 끝에 !를 붙이도록 람다를 만들기
- 예: hello -> Hello!

In [25]:
(lambda word: word.capitalize() + '!')('hello woori')

'Hello woori!'

---
### 4. 제너레이터
- return -> yield
- 시퀀스를 순회할 때 그 시퀀스를 생성하는 객체.
- 한 번 사용되고 사라짐 => 메모리 효율 좋음.

In [26]:
def print_number(num):
    for i in range(num):
        yield i
        
# generator 생성
fx = print_number(5)

# 객체 순회
for i in fx:
    print(i)

0
1
2
3
4


In [27]:
# 이미 메모리에서 사라짐, 아무것도 안 뜸!
for i in fx:
    print(i)

#### `실습4`
- range() 구현하기
- 제너레이터 사용
- 
```
def my_range(start, end, step):
    yield
```

In [28]:
def my_range(start, end, step=1):
    while start < end:
        yield start
        start += step
    
ranger = my_range(-5, 6, 2)
for i in ranger:
    print(i)

-5
-3
-1
1
3
5


In [29]:
# generator 더 쉽게 만들기
# 리스트 컴프리헨션처럼 만들고, []가 아닌 ()로 담아주면 됨!

ranger = (i for i in range(5))
print(type(ranger))
print()

for i in ranger:
    print(i)

<class 'generator'>

0
1
2
3
4


---
### 5. 재귀함수
- 너무 깊으면 예외 발생 => !주의!
- 자기 자신을 호출하는 함수
- [[1, 2, 3], [[1, 1]], 4, 5] -> [1, 2, 3, 1, 1, 4, 5]
- 차원이 다른 모든 요소를 하나의 리스트에 담고 싶을 때

In [30]:
isinstance('h', int) # 'h'가 int인가? t/f 반환

False

In [31]:
def flatten(sent):
    for word in sent:
        if isinstance(word, list):
            # true
            for sub_word in flatten(word):
                yield sub_word
        else:
            # false
            yield word

In [32]:
a = [[4, 5, 6], [[1, [2, 3]]], 7, 8]
flatten(a)

for i in flatten(a):
    print(i)

4
5
6
1
2
3
7
8


In [33]:
# python 3.3부터 가능한 기능
def flatten(sent):
    for word in sent:
        if isinstance(word, list):
            # for sub_word in flatten(word):
            #     yield sub_word
            yield from flatten(word) # 위 두 줄의 코드를 한 줄로 축약해서 사용 가능!
        else:
            yield word

---
---
## 예외 처리 | exception handling
1. 예외 처리
2. 예외 발생시키기
     1. raise
     2. assert
3. 예외 정의하기

---
### 1. 예외 처리
- 목적은 프로그램를 정상적으로 종료하기 위함.
- 예외 발생 시, 사용자(우리)에게 알리고 조치를 취함.
- 소프트렌딩
- 형식?
```
try:
    <에러 발생될 법한 코드 블럭>
except <에러 타입>:
    <처리할 방법>
```

In [34]:
1 / 0     # 1. ZeroDivisionError

ZeroDivisionError: division by zero

In [35]:
int('woori')     # 2. ValueError

ValueError: invalid literal for int() with base 10: 'woori'

In [36]:
woori += 1     # 3. NameError

NameError: name 'woori' is not defined

In [37]:
'woori'[10]     # 4. IndexError

IndexError: string index out of range

In [38]:
try:
    # <에러 발생될 법한 코드 블럭>
    1 / 0
except ZeroDivisionError: # <에러 타입>
    # <처리할 방법>
    print('0으로 나눌 수 없습니다.')

0으로 나눌 수 없습니다.


In [39]:
# 바로 error만 프린트하고, 끝남
# exception을 잡는 위치가 중요함!

try:
    for i in range(10):
        print(10 / i)
except ZeroDivisionError:
    print('error')

error


In [40]:
# 이렇게 수정하면, 0일 때 빼고는 돌아간다!

for i in range(10):
    try:
        print(10 / i)
    except ZeroDivisionError:
        print('error')

error
10.0
5.0
3.3333333333333335
2.5
2.0
1.6666666666666667
1.4285714285714286
1.25
1.1111111111111112


In [41]:
# input 받을 때, error 잡기!

word = 'hello woori'
while True:
    index = input('인덱스를 입력하세요. q)종료 >>> ')
    if index == 'q':
        break
        
    try:
        print(word[int(index)])
    except IndexError as e1:
        print('index error')
        print(e1)
    except ValueError as e2:
        print('type error')
        print(e2)

인덱스를 입력하세요. q)종료 >>>  100


index error
string index out of range


인덱스를 입력하세요. q)종료 >>>  우리


type error
invalid literal for int() with base 10: '우리'


인덱스를 입력하세요. q)종료 >>>  7


o


인덱스를 입력하세요. q)종료 >>>  q


---
### 2. 예외 발생시키기
- 프로그램을 강제 종료하고자 할 때 사용


#### A. raise
```
raise ValueError('print ...')
```

In [42]:
while True:
    num = input('number >> ')
    # 숫자가 아니면, error를 발생시키겠다!
    if not num.isdigit():
        raise ValueError('숫자가 아닙니다.')
    else:
        print(num)
        break

number >>  우리


ValueError: 숫자가 아닙니다.

#### B. assert
```
assert <참인 조건>, '예외 메시지' # AssertionError
```

In [43]:
def get_binary(num):
    # 어디에서 에러가 뜨는지 내가 확인할 수 있음, *체크의 기능*
    assert isinstance(num, int), '정수가 아닙니다.'
    return bin(num)

get_binary('woori')

AssertionError: 정수가 아닙니다.

---
### 3. 예외 정의하기
- 사용자 정의 예외
- Exception이라는 부모 클래스를 상속받아야 한다.

In [44]:
# 내용은 안 써도 되는데, 꼭 Exception 클래스를 상속 받아야 함!
class MyException(Exception):
    pass

In [45]:
# 방법 1.
for word in ['A', 'B', 'c', 'D']:
    if word.islower():
        raise MyException('소문자 안 됨!')
    else:
        print(word)

A
B


MyException: 소문자 안 됨!

In [46]:
# 방법 2.
class LowercaseMyException(Exception):
    def __init__(self):
        super().__init__('소문자 안 됨!!!!!')
        
        
for word in ['A', 'B', 'c', 'D']:
    if word.islower():
        raise LowercaseMyException
    else:
        print(word)

A
B


LowercaseMyException: 소문자 안 됨!!!!!